# LSTM 

## introduction

In this notebook i am using a lstm to make a model that can predict the crop yield based on the weather data. Since the data is high resolution being 24 rows per day i am using the mean of the data to make it daily data. Since the growing season for pecans is January - December there is no real need to remove any of the data. I will then use the lstm to predict the crop yield. I then need to normalize all the data since there is a large variation between different features. I will then use the data to train the lstm model. I will then use the model to predict the crop yield. I will then use the mean squared error to evaluate the model.

In [2]:
# load libraries
import lstm
import time
import torch
import queue
import threading
import numpy as np 
import pandas as pd
import torch.nn as nn
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset, random_split

print("Libraries loaded successfully")

Libraries loaded successfully


## Load and check the data

In [3]:
# load data

df = pd.read_csv('../../../data/pecan/combined_05_23_24.csv')

#analyze the data
def display_info(df):
    print(f'df shape: {df.shape}')
    print(f'df columns: {df.columns}')
    print(f'df dtypes: \n{df.dtypes}')
    return df.head()

display_info(df)

df shape: (192840, 17)
df columns: Index(['city_name', 'lat', 'lon', 'temp', 'dew_point', 'feels_like',
       'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed',
       'wind_deg', 'clouds_all', 'weather_main', 'weather_description', 'date',
       'yield'],
      dtype='object')
df dtypes: 
city_name               object
lat                    float64
lon                    float64
temp                   float64
dew_point              float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure               float64
humidity               float64
wind_speed             float64
wind_deg               float64
clouds_all             float64
weather_main           float64
weather_description    float64
date                    object
yield                  float64
dtype: object


,city_name,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,weather_description,date,yield
0,Doña Ana,32.39308,-106.81578,8.99,1.42,7.93,7.60,9.46,1019.0,59.0,2.10,20.0,90.0,1.0,16.0,2016-01-01 00:00:00+00:00,1837.0
1,Doña Ana,32.39308,-106.81578,8.25,2.07,7.63,7.44,8.68,1020.0,65.0,1.52,51.0,87.0,1.0,16.0,2016-01-01 01:00:00+00:00,1837.0
2,Doña Ana,32.39308,-106.81578,7.37,1.44,6.04,6.10,7.80,1021.0,66.0,2.10,340.0,81.0,1.0,0.0,2016-01-01 02:00:00+00:00,1837.0
3,Doña Ana,32.39308,-106.81578,5.58,1.32,3.13,4.30,6.02,1022.0,74.0,3.10,350.0,40.0,1.0,21.0,2016-01-01 03:00:00+00:00,1837.0
4,Doña Ana,32.39308,-106.81578,5.43,0.79,5.43,4.75,5.90,1023.0,72.0,1.17,48.0,87.0,1.0,16.0,2016-01-01 04:00:00+00:00,1837.0


In [4]:
# city name is not needed
df = df.drop('city_name', axis=1)

## Aggregate the data

In [5]:
df['date'] = pd.to_datetime(df['date'])
df = df.groupby(df['date'].dt.date).mean()
df = df.drop(columns=['date'], errors='ignore')
df.reset_index(inplace=True)

In [6]:
display_info(df)

df shape: (2557, 16)
df columns: Index(['date', 'lat', 'lon', 'temp', 'dew_point', 'feels_like', 'temp_min',
       'temp_max', 'pressure', 'humidity', 'wind_speed', 'wind_deg',
       'clouds_all', 'weather_main', 'weather_description', 'yield'],
      dtype='object')
df dtypes: 
date                    object
lat                    float64
lon                    float64
temp                   float64
dew_point              float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure               float64
humidity               float64
wind_speed             float64
wind_deg               float64
clouds_all             float64
weather_main           float64
weather_description    float64
yield                  float64
dtype: object


,date,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,weather_description,yield
0,2016-01-01,32.93441,-105.644055,1.392083,-1.882708,-1.374167,0.882500,1.950625,1032.041667,78.187500,2.702292,92.895833,90.708333,2.583333,15.312500,1780.5
1,2016-01-02,32.93441,-105.644055,0.836042,-2.313125,-0.411667,-0.019375,1.648333,1032.250000,79.375000,1.234167,154.583333,78.125000,1.229167,15.500000,1780.5
2,2016-01-03,32.93441,-105.644055,0.148750,-4.325417,-0.972292,-0.720417,0.944792,1028.812500,72.458333,1.148125,121.687500,20.000000,1.229167,19.958333,1780.5
3,2016-01-04,32.93441,-105.644055,1.927708,-4.207708,0.319167,0.775208,2.791250,1025.479167,63.479167,2.094167,143.041667,42.229167,0.541667,19.854167,1780.5
4,2016-01-05,32.93441,-105.644055,3.300417,0.621875,1.956042,2.570833,3.927917,1021.270833,82.458333,1.621458,82.270833,93.500000,3.458333,14.770833,1780.5


In [7]:
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop('date', axis=1)

display_info(df)

df shape: (2557, 18)
df columns: Index(['lat', 'lon', 'temp', 'dew_point', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all',
       'weather_main', 'weather_description', 'yield', 'year', 'month', 'day'],
      dtype='object')
df dtypes: 
lat                    float64
lon                    float64
temp                   float64
dew_point              float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure               float64
humidity               float64
wind_speed             float64
wind_deg               float64
clouds_all             float64
weather_main           float64
weather_description    float64
yield                  float64
year                     int32
month                    int32
day                      int32
dtype: object


,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,weather_description,yield,year,month,day
0,32.93441,-105.644055,1.392083,-1.882708,-1.374167,0.882500,1.950625,1032.041667,78.187500,2.702292,92.895833,90.708333,2.583333,15.312500,1780.5,2016,1,1
1,32.93441,-105.644055,0.836042,-2.313125,-0.411667,-0.019375,1.648333,1032.250000,79.375000,1.234167,154.583333,78.125000,1.229167,15.500000,1780.5,2016,1,2
2,32.93441,-105.644055,0.148750,-4.325417,-0.972292,-0.720417,0.944792,1028.812500,72.458333,1.148125,121.687500,20.000000,1.229167,19.958333,1780.5,2016,1,3
3,32.93441,-105.644055,1.927708,-4.207708,0.319167,0.775208,2.791250,1025.479167,63.479167,2.094167,143.041667,42.229167,0.541667,19.854167,1780.5,2016,1,4
4,32.93441,-105.644055,3.300417,0.621875,1.956042,2.570833,3.927917,1021.270833,82.458333,1.621458,82.270833,93.500000,3.458333,14.770833,1780.5,2016,1,5


## Normalize the data

In [8]:
scaler = MinMaxScaler()

yield_df = df['yield']
df = df.drop('yield', axis=1)
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
# df.to_csv('../../../data/pecan/normalized_weather_05_26_24.csv', index=False)
display_info(df)

df shape: (2557, 17)
df columns: Index(['lat', 'lon', 'temp', 'dew_point', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'humidity', 'wind_speed', 'wind_deg', 'clouds_all',
       'weather_main', 'weather_description', 'year', 'month', 'day'],
      dtype='object')
df dtypes: 
lat                    float64
lon                    float64
temp                   float64
dew_point              float64
feels_like             float64
temp_min               float64
temp_max               float64
pressure               float64
humidity               float64
wind_speed             float64
wind_deg               float64
clouds_all             float64
weather_main           float64
weather_description    float64
year                   float64
month                  float64
day                    float64
dtype: object


,lat,lon,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,weather_description,year,month,day
0,0.718798,0.98755,0.197174,0.460090,0.214945,0.206540,0.190260,0.855460,0.819764,0.153835,0.075982,0.907083,0.318493,0.439759,0.0,0.0,0.000000
1,0.718798,0.98755,0.183798,0.448994,0.237162,0.184969,0.182947,0.860814,0.833080,0.012551,0.348466,0.781250,0.151541,0.450602,0.0,0.0,0.033333
2,0.718798,0.98755,0.167266,0.397118,0.224221,0.168201,0.165925,0.772484,0.755519,0.004270,0.203160,0.200000,0.151541,0.708434,0.0,0.0,0.066667
3,0.718798,0.98755,0.210058,0.400153,0.254032,0.203974,0.210598,0.686831,0.654830,0.095313,0.297485,0.422292,0.066781,0.702410,0.0,0.0,0.100000
4,0.718798,0.98755,0.243078,0.524657,0.291816,0.246923,0.238099,0.578694,0.867656,0.049822,0.029049,0.935000,0.426370,0.408434,0.0,0.0,0.133333


## Shape the data to be of the sequence length that we want.

In [9]:
X = torch.tensor(df.values, dtype=torch.float32)
Y = torch.tensor(yield_df.values, dtype=torch.float32)

sequence_length = 32  # xrbitrary number chosen


def create_sequences(X, Y, seq_length):
    xs, ys = [], []
    for i in range(len(X) - seq_length):
        x = X[i:i+seq_length]
        y = Y[i+seq_length]
        xs.append(x)
        ys.append(y)
    return torch.stack(xs), torch.stack(ys)

X, y = create_sequences(X, Y, sequence_length)


print(X.shape, y.shape) 


torch.Size([2525, 32, 17]) torch.Size([2525])


## Prepare the datasets

In [10]:
dataset = TensorDataset(X, y)

torch.save(dataset, '../../../data/pecan/lstm_weather_dataset.pt')
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
new_val_size = int(0.8 * val_size)
test_size = val_size - new_val_size

generator = torch.Generator().manual_seed(42)

train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=generator)
val_dataset, test_dataset = random_split(val_dataset, [new_val_size, test_size], generator=generator)

print(f'Train Dataset size: {len(train_dataset)} \nValidation Dataset size {len(val_dataset)}\nTest Dataset size {len(test_dataset)}')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Train Dataset size: 2020 
Validation Dataset size 404
Test Dataset size 101


## Visualize the data

In [16]:
graph_df = df.rename(columns={
    'lat': 'Lat',
    'lon': 'Long',
    'temp': 'Temp',
    'dew_point': 'Dew Point',
    'feels_like': 'Feels Like',
    'temp_min': 'Min Temp',
    'temp_max': 'Max Temp',
    'pressure': 'Pressure',
    'humidity': ' Humidity',
    'wind_speed': 'Wind Speed',
    'wind_deg': 'Wind Direction',
    'clouds_all': 'Cloud Cover',
    'weather_main': 'Condition',
    'weather_description': 'Description',
    'year': 'Year',
    'month': 'Month',
    'day': 'Day',
})
graph_df = pd.concat([graph_df, yield_df], axis=1)
graph_df = graph_df.rename(columns={'yield' : 'Yield'})

dimensions = ['Lat', 'Long', 'Temp', 'Dew Point', 'Feels Like', 'Min Temp',
       'Max Temp', 'Pressure', ' Humidity', 'Wind Speed', 'Wind Direction',
       'Cloud Cover', 'Condition', 'Description',]


fig = px.parallel_coordinates(graph_df, 
                            color='Yield', 
                            dimensions=dimensions,
                            # color_continuous_scale=colorscale,
                            title='Microclimate Features',
                            color_continuous_scale=px.colors.diverging.Temps)

fig.update_layout(paper_bgcolor='#eceff4', width=1150, height=400)
fig.update_layout(title_x=0.5) 
# fig.write_html('../../../data/pecan/parallel_coordinates/weather_graph.html')
fig.write_html('trash.html')
fig.show()


# Train the model

In [ ]:
# set device 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device set to:", device)

model = lstm.LSTM().to(device)
trainer = lstm.Trainer(model, train_loader, val_loader, device)
model, train_losses, val_losses = trainer.train()

In [ ]:
trace1 = go.Scatter(x=np.arange(len(train_losses)), y=train_losses, mode='lines', name='Train Loss')
trace2 = go.Scatter(x=np.arange(len(val_losses)), y=val_losses, mode='lines', name='Validation Loss')
df = [trace1, trace2]
layout = go.Layout(title='Train vs Validation Loss MSE', xaxis=dict(title='Epochs'), yaxis=dict(title='Loss'))
fig = go.Figure(data=df, layout=layout)
fig.show()

# Running the code block below will run a single thread grid search accross 864 different models